## Finetuning Flan T5 XL for lease agreement data extraction

### Install dependencies

In [1]:
%pip install pandas numpy peft scikit-learn transformers datasets torch accelerate bitsandbytes huggingface-hub -q

Note: you may need to restart the kernel to use updated packages.


### Build the dataframe
Build the dataframe from the collected data

In [2]:
import pandas as pd
import sqlite3

# Path to the SQLite database
db_path = "../output/extracted_lease_agreements.db"

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Query to select all data from the extracted_data table
query = "SELECT * FROM extracted_data"

# Read the data into a DataFrame
df = pd.read_sql_query(query, conn, index_col="id")

# Close the database connection
conn.close()

df.head()

,extracted_text,extracted_fields
id,,
1,"Name, address and phone number of managing age...","{'tenant_name': 'Yolanda Strobert', 'unit_addr..."
2,dupusit: or 2) return the remaining portion (i...,"{'tenant_name': 'Comunque Bolas', 'unit_addres..."
3,"You'll pay for all other utilities, related de...","{'tenant_name': None, 'unit_address': None, 'u..."
4,APARTMENT LEASE CONTRACT\nNAA NATIONAL APARTME...,"{'tenant_name': 'Dominique Boles', 'unit_addre..."
5,Disposition or Sale. Except for animals and pr...,"{'tenant_name': None, 'unit_address': None, 'u..."


### Split the dataset

In [3]:
from sklearn.model_selection import train_test_split

# First split: 80% for training, 20% for temp (which will be split into eval and test)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)

# Second split: 50% of temp for eval and 50% for test (10% of the original data each)
eval_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Check the sizes of the splits
print(f"Training set size: {len(train_df)}")
print(f"Evaluation set size: {len(eval_df)}")
print(f"Test set size: {len(test_df)}")


Training set size: 205
Evaluation set size: 26
Test set size: 26


### Load the base model

In [4]:
from peft import LoraConfig, TaskType

peft_config = LoraConfig(
    lora_alpha=16, # Higher alpha to match larger model capacity
    lora_dropout=0.1, # Consistent dropout rate to prevent overfitting
    r=8, # Rank, kept the same for balance between performance and efficiency
    task_type=TaskType.SEQ_2_SEQ_LM, # Change to SEQ_2_SEQ_LM for seq2seq models
    bias="none", # Keeping bias as none, similar to your original setup
    target_modules=[
        'q', 'v', 'k', 'o', # Attention layers (query, value, key, output projections)
        'wi', 'wo', # Feedforward layers (input, output projections)
        'wq', 'wv', 'wk', 'wo', # Additional T5-specific projection layers
    ], # Target modules relevant to T5 architecture
)


In [5]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
from huggingface_hub import notebook_login
from peft import get_peft_model, prepare_model_for_kbit_training
import torch

# login to access gated model
notebook_login()

# Load the tokenizer and model
model_id = "google/flan-t5-xl"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# empty GPU memory
torch.cuda.empty_cache()

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

model = AutoModelForSeq2SeqLM.from_pretrained(model_id, quantization_config=bnb_config)  # Change num_labels based on your task

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


/home/aryaniyaps/velocius-ai/leaseaudit-model-training/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 12,189,696 || all params: 2,861,946,880 || trainable%: 0.4259


### Build the datasets

In [6]:
extraction_json_schema = {
  "title": "ExtractFieldsResult",
  "type": "object",
  "properties": {
    "tenant_name": {
      "type": ["string", "null"],
      "description": "The name of the tenant, found in the OCR text."
    },
    "unit_address": {
      "type": ["string", "null"],
      "description": "The unit address found in the OCR text."
    },
    "unit_number": {
      "type": ["string", "null"],
      "description": "The unit number found in the OCR text."
    },
    "unit_type": {
      "type": ["string", "null"],
      "description": "The unit type found in the OCR text."
    },
    "agreement_date": {
      "type": ["string", "null"],
      "format": "date"
    },
    "lease_start": {
      "type": ["string", "null"],
      "format": "date",
      "description": "The date when the lease starts, found in the OCR text."
    },
    "lease_end": {
      "type": ["string", "null"],
      "format": "date",
      "description": "The date when the lease ends, found in the OCR text."
    },
    "lease_auto_renew": {
      "type": ["string", "null"],
      "description": "The type of lease auto renewal, found in the OCR text."
    },
    "hourly_rate": {
      "type": ["number", "null"],
      "description": "The hourly rate found in the OCR text."
    },
    "monthly_rent": {
      "type": ["number", "null"],
      "description": "The monthly rent found in the OCR text."
    },
    "prorated_rent": {
      "type": ["number", "null"],
      "description": "The prorated rent found in the OCR text."
    },
    "security_deposit": {
      "type": ["number", "null"],
      "description": "The security deposit found in the OCR text."
    },
    "lease_rent": {
      "type": ["number", "null"],
      "description": "The security deposit found in the OCR text."
    },
    "monthly_payment_breakdown": {
      "type": ["object", "null"],
      "description": "The monthly payment breakdown data found in the OCR text.",
      "additionalProperties": {}
    },
    "utility_charges": {
      "type": ["object", "null"],
      "description": "The utility charges found in the OCR text. This is a dictionary with utility charges as the key, and their price as the value.",
      "additionalProperties": {
        "type": ["number", "null"]
      }
    }
  },
  "required": []
}


In [7]:
from datasets import Dataset

prompt_template = """
### TASK:
You are a data extraction model designed to parse and extract specific information from text. Your task is to extract relevant data fields from the following lease agreement document provided as OCR text.

### INPUT TEXT:
Below is the OCR text from a lease agreement. Use this text to extract the required data fields.

OCR Text:
{extracted_text}

### RESPONSE FORMAT:
The extracted data should be in JSON, adhering to the JSON schema below:

```json
{extraction_json_schema}
```

### REQUIRED OUTPUT:
Provide the extracted data in JSON format, strictly following the schema above.

### RESPONSE:        
"""

def preprocess_function(examples):
    inputs = [prompt_template.format(extracted_text=extracted_text, extraction_json_schema=extraction_json_schema) for extracted_text in examples["extracted_text"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    # The "labels" are the tokenized outputs:
    labels = tokenizer(
        text_target=examples["extracted_fields"], 
        max_length=512,         
        truncation=True
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Create datasets with the extracted text and labels
train_dataset = Dataset.from_pandas(train_df)
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = Dataset.from_pandas(eval_df)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)
test_dataset = Dataset.from_pandas(test_df)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

train_dataset


Map:   0%|          | 0/205 [00:00<?, ? examples/s]

Map:   0%|          | 0/26 [00:00<?, ? examples/s]

Map:   0%|          | 0/26 [00:00<?, ? examples/s]

Dataset({
    features: ['extracted_text', 'extracted_fields', 'id'],
    num_rows: 205
})

### Train (fine tune) the base model

In [8]:
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="steps",  # Changed to evaluate based on steps
    eval_steps=1,  # Evaluate every 1 step
    learning_rate=1e-3,
    lr_scheduler_type="linear",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    logging_strategy="steps",
    logging_steps=2,
    disable_tqdm=False,  # Ensure tqdm progress bar is enabled
    max_steps = 10,
    use_cpu=True,
    save_strategy="steps",
    save_steps=1,  # Save a checkpoint every 1 step
    # predict_with_generate=True, # need this for ROUGE/ BLEU metrics
    optim="paged_adamw_32bit",
)

trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    args=training_args,
    data_collator=DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        model=model,
        return_tensors="pt",  # Return PyTorch tensors
    )
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

max_steps is given, it will override any value given in num_train_epochs
/home/aryaniyaps/velocius-ai/leaseaudit-model-training/.venv/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/aryaniyaps/velocius-ai/leaseaudit-model-training/.venv/lib/python3.12/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
1,No log,1.282624
2,1.433800,1.145782
3,1.433800,1.035312
4,1.246500,0.938740
5,1.246500,0.851025
6,1.069000,0.777541
7,1.069000,0.720084
8,0.934500,0.678166
9,0.934500,0.649756
10,0.787200,0.635124


/home/aryaniyaps/velocius-ai/leaseaudit-model-training/.venv/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/aryaniyaps/velocius-ai/leaseaudit-model-training/.venv/lib/python3.12/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/home/aryaniyaps/velocius-ai/leaseaudit-model-training/.venv/lib/python3.12/site-packages/torch/_dynamo/e

TrainOutput(global_step=10, training_loss=1.0941948771476746, metrics={'train_runtime': 19618.0545, 'train_samples_per_second': 0.016, 'train_steps_per_second': 0.001, 'total_flos': 2722954752294912.0, 'train_loss': 1.0941948771476746, 'epoch': 1.5384615384615383})

### Save the model

In [9]:
trainer.model.save_pretrained("./saved_models/finetuned_flan_t5_xl_lease_data_extraction")

### Evaluate the model

In [15]:
from peft import PeftModel, PeftConfig
 
# Load peft config for pre-trained checkpoint etc.
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

model = AutoModelForSeq2SeqLM.from_pretrained(model_id, quantization_config=bnb_config)

peft_model_id = "./saved_models/finetuned_flan_t5_xl_lease_data_extraction"
config = PeftConfig.from_pretrained(peft_model_id)
 
# Load the Lora model
finetuned_model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
finetuned_model.eval()
 
print("Peft model loaded")

/home/aryaniyaps/velocius-ai/leaseaudit-model-training/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Peft model loaded


Let's try data extraction with a random sample

In [18]:
from random import randrange

sample = test_dataset[randrange(len(test_dataset))]
 
input_ids = tokenizer(prompt_template.format(extracted_text=sample["extracted_text"], extraction_json_schema=extraction_json_schema), return_tensors="pt", truncation=True).input_ids.cuda()
with torch.inference_mode():
    outputs = finetuned_model.generate(input_ids=input_ids, max_new_tokens=512, do_sample=True)
print(f"input OCR TEXT: {sample['extracted_text'][-75:]}\n{'---'* 20}")
 
print(f"Extracted data:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]}")

input OCR TEXT: concession amount of $3,350 will immediately become due back to Landlord.
2
------------------------------------------------------------
Extracted data:
'tenancy period': None, 'leaselength': None, 'leasepayment': None, 'leasepayment': None, 'leaserent': None, 'leasepayment': None, 'leasepayment': None, 'leasesettlement': None, 'leaserenewal': None, 'leaseupdate': None


### Evaluate the model
(Using F1 score and exact matches)

#### Install dependencies

In [33]:
%pip install evaluate tabulate -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [24]:
import evaluate
from tqdm import tqdm

# Metrics
f1_metric = evaluate.load("f1")
exact_match_metric = evaluate.load("accuracy")

def evaluate_peft_model(sample, max_target_length=512):
    # Generate extracted data
    input_ids = torch.tensor(sample["input_ids"]).unsqueeze(0).cuda()
    outputs = model.generate(input_ids=input_ids, do_sample=True, max_new_tokens=max_target_length)
    prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)
    
    # Decode eval sample
    labels = tokenizer.decode(sample['labels'], skip_special_tokens=True)
 
    return prediction, labels

# Run predictions
predictions, references = [], []
for sample in tqdm(tokenized_test_dataset):
    prediction, reference = evaluate_peft_model(sample)
    predictions.append(prediction)
    references.append(reference)



100%|██████████| 26/26 [18:11<00:00, 41.97s/it]


ValueError: Predictions and/or references don't match the expected format.
Expected format: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)},
Input predictions: [["'", 'l', 'e', 'a', 's', 'e', '_', 's', 't', 'a', 'r', 't', '_', 'd', 'a', 't', 'e', "'", ':', ' ', 'f', 'a', 'l', 's', 'e', ',', ' ', "'", 'l', 'e', 'a', 's', 'e', '_', 'e', 'n', 'd', '_', 'd', 'a', 't', 'e', "'", ':', ' ', 'f', 'a', 'l', 's', 'e', ',', ' ', "'", 'l', 'e', 'a', 's', 'e', '_', 'l', 'e', 'n', 'g', 't', 'h', '_', 's', 't', 'a', 'r', 't', "'", ':', ' ', 'f', 'a', 'l', 's', 'e', ',', ' ', "'", 'l', 'e', 'a', 's', 'e', '_', 'l', 'e', 'n', 'g', 't', 'h', '_', 'e', 'n', 'd', "'", ':', ' ', 'f', 'a', 'l', 's', 'e', ',', ' ', "'", 'l', 'e', 'a', 's', 'e', '_', 'l', 'e', 'n', 'g', 't', 'h', '_', 's', 't', 'a', 'r', 't', "'", ':', ' ', 'f', 'a', 'l', 's', 'e', ',', ' ', "'", 'l', 'e', 'a', 's', 'e', '_', 'l', 'e', 'n', 'g', 't', 'h', '_', 'e', 'n', 'd', "'", ':', ' ', 'f', 'a',

In [38]:
from tabulate import tabulate

# Prepare data for tabulation
table_data = []
for i, (pred, ref) in enumerate(zip(predictions, references), 1):
    table_data.append([i, ref, pred])

# Define table headers
headers = ["#", "Reference", "Prediction"]

# Display the table
print(tabulate(table_data, headers=headers, tablefmt="fancy_grid", showindex=False))


╒═════╤═══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

In [ ]:
# TODO: calculate accuracy using metrics